In [171]:
# Import Modules
import pandas as pd
import numpy as np

import datetime
import dateparser
from datetime import date, timedelta

# Import Modules
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

import sys
# adding Notebooksfolder to the system path
sys.path.insert(0, '/Users/emil/miniforge3/envs/googleapi/Notebooks')

import importlib
import dates_funcs
importlib.reload(dates_funcs)
from dates_funcs import appendDFToCSV


In [20]:
sys.path

['/Users/emil/miniforge3/envs/googleapi/Notebooks',
 '/Users/emil/miniforge3/envs/googleapi/Notebooks',
 '/Users/emil/miniforge3/envs/googleapi/Notebooks/Search_Console',
 '/Users/emil/miniforge3/envs/googleapi/lib/python311.zip',
 '/Users/emil/miniforge3/envs/googleapi/lib/python3.11',
 '/Users/emil/miniforge3/envs/googleapi/lib/python3.11/lib-dynload',
 '',
 '/Users/emil/miniforge3/envs/googleapi/lib/python3.11/site-packages']

In [21]:
# Function to fully display dataframe
def force_show_all(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', None):
        display(HTML(df.to_html()))
# ... now when you're ready to fully display df:
#force_show_all(df)

In [22]:
#FRAN https://www.shortautomaton.com/connecting-to-google-search-console-api-with-python/ 

# Define function to get authorization
def gsc_auth(scopes):
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', scopes)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secrets.json', scopes)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('searchconsole', 'v1', credentials=creds)

    return service

scopes = ['https://www.googleapis.com/auth/webmasters.readonly']

service = gsc_auth(scopes)

Query Function

In [41]:
def query_full(service, site_url, payload):
    """Run a query on the Google Search Console API and return a dataframe of results.
    
    Args:
        service (object): Service object from connect()
        site_url (string): URL of Google Search Console property
        payload (dict): API query payload dictionary
    
    Return:
        df (dataframe): Pandas dataframe containing requested data. 
    
    """
    
    response = service.searchanalytics().query(siteUrl=site_url, body=payload).execute()
    
    results = []
    
    for row in response['rows']:    
        data = {}
        
        for i in range(len(payload['dimensions'])):
            data[payload['dimensions'][i]] = row['keys'][i]

        data['clicks'] = row['clicks']
        data['impressions'] = row['impressions']
        data['ctr'] = round(row['ctr'] * 100, 2)
        data['position'] = round(row['position'], 2)        
        results.append(data)
    
    return pd.DataFrame.from_dict(results)

Find latest Date in old CSV

In [24]:
# Find maximum date in csv file.

# Read file in chunks (parts)
df = pd.read_csv("output/csv/testa.csv", chunksize=100000)

# Max date to start comparing to
#max_date_overall = testdateUS.strftime("%Y-%m-%d")
max_date_overall = datetime.datetime(2022, 1, 1).date()

#time.strptime(date1, "%d/%m/%Y")

print(max_date_overall)


for chunk in df:
    chunk['date'] = chunk['date']
    max_date_in_chunk = max(chunk['date'])
    
    max_date_in_chunk = datetime.datetime.strptime(max_date_in_chunk,'%Y-%m-%d').date()
    
    if max_date_in_chunk > max_date_overall:
        max_date_overall = max_date_in_chunk
    
    print('maxdate in chunk: ',max_date_overall) 

test_start_date_name = '2022-01-01'
test_start_date = dateparser.parse(test_start_date_name, settings={'TIMEZONE': 'America/Los_Angeles'}).date()

test_end_date_name = 'three days ago'
test_end_date = dateparser.parse(test_end_date_name, settings={'TIMEZONE': 'America/Los_Angeles'}).date()

print('maxdate in csv: ',max_date_overall) 
print('start date given: ',test_start_date)
print('end date given: ',test_end_date)

start_fetch_date = max_date_overall+datetime.timedelta(days=1)
start_fetch_date = start_fetch_date.strftime("%Y-%m-%d")

if test_start_date > max_date_overall:
    print('start date is after largest date in csv,', end='\n')
    print('will fetch updated data from date: ', start_fetch_date, end='\n')   
    print('to : ', test_end_date)
else:
    print('data already fetched, wait a day before running again')

print(max_date_overall+datetime.timedelta(days=1))

print(start_fetch_date)
print(test_end_date)

FileNotFoundError: [Errno 2] No such file or directory: 'output/csv/testa.csv'

In [147]:
# First Dates with data available

max_date_overall_name = '2022-01-27'
max_date_overall = dateparser.parse(max_date_overall_name, settings={'TIMEZONE': 'America/Los_Angeles'}).date()

test_start_date_name = '2022-01-28'
test_start_date = dateparser.parse(test_start_date_name, settings={'TIMEZONE': 'America/Los_Angeles'}).date()

test_end_date_name = 'three days ago'
test_end_date = dateparser.parse(test_end_date_name, settings={'TIMEZONE': 'America/Los_Angeles'}).date()

LOOPA TEST!

In [148]:
# Loop to send query between start_date and end_date-1 day. Each date to a new DF in dfs. 
# Then concatenated into final DF.



def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

# Only run if dates are not already in old csv.        
        
if test_start_date > max_date_overall:        
        
    start_date = max_date_overall+datetime.timedelta(days=1)
    end_date = test_end_date

    dfs = []

    for query_date in daterange(start_date, end_date):
        query_date = query_date.strftime("%Y-%m-%d")

        payload = {
            'startDate': query_date,
            'endDate': query_date,
            'dimensions': ["date","page","device","query","country"],
            'rowLimit': 25000,
            'startRow': 0
            }

        site_url = "https://advokatfamiljforsvar.se"

        df = query_full(service, site_url, payload)

        dfs.append(df)

    # New DF med all data fran start_date - end_data-1 
    df_new = pd.concat(dfs, ignore_index=True)
    df_new.reset_index(inplace=True, drop=True)

    # Sort columns in same order as the file to write to
    df_new.sort_values(by=["date","page","device","query","country"],ascending=True,inplace=True)

    # Columns to case matching reference and analytics data
    df_new['device'] = df_new['device'].str.lower()
    df_new['country'] = df_new['country'].str.upper()

    # Column names to matching reference and analytics data
    df_new.rename(columns = {'date':'date','country':'countryIdLong', 'page':'landingPage', 'device':'deviceCategory'}, inplace = True)

    
    # To File
    csvFilePath = "output/csv/testa.csv"
    sep = ","

    dates_funcs.appendDFToCSV(df, csvFilePath, fileName, sep)

else:
    
    print('Wait a day, before running again')
    #display(df_new.tail())

NameError: name 'df_nwq' is not defined

In [156]:
df_test = df_new.copy()

In [157]:
df_test

,date,landingPage,deviceCategory,query,countryIdLong,clicks,impressions,ctr,position
3,2022-01-28,https://advokatfamiljforsvar.se/,desktop,advokat specialist familjerätt,TWN,0,1,0.00,58.0
0,2022-01-28,https://advokatfamiljforsvar.se/,desktop,advokatbyrån familj och försvar,SWE,2,3,66.67,1.0
1,2022-01-28,https://advokatfamiljforsvar.se/,desktop,familj och försvar,SWE,1,2,50.00,1.0
2,2022-01-28,https://advokatfamiljforsvar.se/,mobile,familj och försvar,SWE,1,1,100.00,1.0
4,2022-01-28,https://advokatfamiljforsvar.se/,mobile,retorik advokater,USA,0,1,0.00,18.0
...,...,...,...,...,...,...,...,...,...
753918,2023-06-08,https://advokatfamiljforsvar.se/verkstallighet...,desktop,verkställighet,USA,0,2,0.00,25.5
753919,2023-06-08,https://advokatfamiljforsvar.se/verkstallighet...,mobile,umgängessabotage,NZL,0,1,0.00,36.0
753920,2023-06-08,https://advokatfamiljforsvar.se/verkstallighet...,mobile,vad är verkställighet,SWE,0,1,0.00,12.0
753921,2023-06-08,https://advokatfamiljforsvar.se/verkstallighet...,mobile,verkställighet,DEU,0,1,0.00,26.0


In [158]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 753923 entries, 3 to 753922
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   date            753923 non-null  object 
 1   landingPage     753923 non-null  object 
 2   deviceCategory  753923 non-null  object 
 3   query           753923 non-null  object 
 4   countryIdLong   753923 non-null  object 
 5   clicks          753923 non-null  int64  
 6   impressions     753923 non-null  int64  
 7   ctr             753923 non-null  float64
 8   position        753923 non-null  float64
dtypes: float64(2), int64(2), object(5)
memory usage: 57.5+ MB


In [159]:
df_sorted = dates_funcs.sortOut_date_searchFormat(df_test)

In [160]:
df_sorted

,dateFull,year,quarter,month,monthName,week,weekday,weekdayName,landingPage,deviceCategory,query,countryIdLong,clicks,impressions,ctr,position
3,2022-01-28,2022,1,1,January,4,5,Friday,https://advokatfamiljforsvar.se/,desktop,advokat specialist familjerätt,TWN,0,1,0.00,58.0
0,2022-01-28,2022,1,1,January,4,5,Friday,https://advokatfamiljforsvar.se/,desktop,advokatbyrån familj och försvar,SWE,2,3,66.67,1.0
1,2022-01-28,2022,1,1,January,4,5,Friday,https://advokatfamiljforsvar.se/,desktop,familj och försvar,SWE,1,2,50.00,1.0
2,2022-01-28,2022,1,1,January,4,5,Friday,https://advokatfamiljforsvar.se/,mobile,familj och försvar,SWE,1,1,100.00,1.0
4,2022-01-28,2022,1,1,January,4,5,Friday,https://advokatfamiljforsvar.se/,mobile,retorik advokater,USA,0,1,0.00,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
753918,2023-06-08,2023,2,6,June,23,4,Thursday,https://advokatfamiljforsvar.se/verkstallighet...,desktop,verkställighet,USA,0,2,0.00,25.5
753919,2023-06-08,2023,2,6,June,23,4,Thursday,https://advokatfamiljforsvar.se/verkstallighet...,mobile,umgängessabotage,NZL,0,1,0.00,36.0
753920,2023-06-08,2023,2,6,June,23,4,Thursday,https://advokatfamiljforsvar.se/verkstallighet...,mobile,vad är verkställighet,SWE,0,1,0.00,12.0
753921,2023-06-08,2023,2,6,June,23,4,Thursday,https://advokatfamiljforsvar.se/verkstallighet...,mobile,verkställighet,DEU,0,1,0.00,26.0


In [164]:
df_by_years = dates_funcs.split_years(df_sorted)

In [165]:
df_by_years[1]

,dateFull,year,quarter,month,monthName,week,weekday,weekdayName,landingPage,deviceCategory,query,countryIdLong,clicks,impressions,ctr,position
437369,2023-01-01,2023,1,1,January,52,7,Sunday,https://advokatfamiljforsvar.se/,desktop,advokat,ARE,0,2,0.0,80.0
437370,2023-01-01,2023,1,1,January,52,7,Sunday,https://advokatfamiljforsvar.se/,desktop,advokat,CIV,0,1,0.0,96.0
437371,2023-01-01,2023,1,1,January,52,7,Sunday,https://advokatfamiljforsvar.se/,desktop,advokat,ESP,0,1,0.0,79.0
437372,2023-01-01,2023,1,1,January,52,7,Sunday,https://advokatfamiljforsvar.se/,desktop,advokat,IND,0,2,0.0,81.5
437373,2023-01-01,2023,1,1,January,52,7,Sunday,https://advokatfamiljforsvar.se/,desktop,advokat,IRQ,0,1,0.0,80.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
753918,2023-06-08,2023,2,6,June,23,4,Thursday,https://advokatfamiljforsvar.se/verkstallighet...,desktop,verkställighet,USA,0,2,0.0,25.5
753919,2023-06-08,2023,2,6,June,23,4,Thursday,https://advokatfamiljforsvar.se/verkstallighet...,mobile,umgängessabotage,NZL,0,1,0.0,36.0
753920,2023-06-08,2023,2,6,June,23,4,Thursday,https://advokatfamiljforsvar.se/verkstallighet...,mobile,vad är verkställighet,SWE,0,1,0.0,12.0
753921,2023-06-08,2023,2,6,June,23,4,Thursday,https://advokatfamiljforsvar.se/verkstallighet...,mobile,verkställighet,DEU,0,1,0.0,26.0


In [166]:
df2022 = df_by_years[0].copy()

In [167]:
df2023 = df_by_years[1].copy()

In [172]:
df_2022_by_month = dates_funcs.split_months(df2022)

In [173]:
df_2023_by_month = dates_funcs.split_months(df2023)

In [182]:
df_2023_by_month[5]

,dateFull,year,quarter,month,monthName,week,weekday,weekdayName,landingPage,deviceCategory,query,countryIdLong,clicks,impressions,ctr,position
734376,2023-06-01,2023,2,6,June,22,4,Thursday,https://advokatfamiljforsvar.se/,desktop,advokat,IND,0,2,0.0,55.5
734377,2023-06-01,2023,2,6,June,22,4,Thursday,https://advokatfamiljforsvar.se/,desktop,advokat,JPN,0,1,0.0,94.0
734378,2023-06-01,2023,2,6,June,22,4,Thursday,https://advokatfamiljforsvar.se/,desktop,advokat,KOR,0,1,0.0,79.0
734379,2023-06-01,2023,2,6,June,22,4,Thursday,https://advokatfamiljforsvar.se/,desktop,advokat,PRI,0,1,0.0,91.0
734380,2023-06-01,2023,2,6,June,22,4,Thursday,https://advokatfamiljforsvar.se/,desktop,advokat,SWE,0,1,0.0,91.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
753918,2023-06-08,2023,2,6,June,23,4,Thursday,https://advokatfamiljforsvar.se/verkstallighet...,desktop,verkställighet,USA,0,2,0.0,25.5
753919,2023-06-08,2023,2,6,June,23,4,Thursday,https://advokatfamiljforsvar.se/verkstallighet...,mobile,umgängessabotage,NZL,0,1,0.0,36.0
753920,2023-06-08,2023,2,6,June,23,4,Thursday,https://advokatfamiljforsvar.se/verkstallighet...,mobile,vad är verkställighet,SWE,0,1,0.0,12.0
753921,2023-06-08,2023,2,6,June,23,4,Thursday,https://advokatfamiljforsvar.se/verkstallighet...,mobile,verkställighet,DEU,0,1,0.0,26.0
